# Notes:

## fix tick marks on distribution to be only at .1 mile marks 

In [ ]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

Objective: Identify route and stop characteristics:

- Speed of route
- Ridership at stops
- Route spacing

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import datetime
import geopandas
import random
import math

In [ ]:
print("GTFS data sets: ")
GTFS_DATA_PATH = "mmt_gtfs/"
import os; os.listdir(GTFS_DATA_PATH) # Peak whats in the directory 

In [ ]:
def getPandasDFCSV(path, file, sep=','):  
    """ Keyword Arg sep: deliminator used in txt file (default = ',')"""
    pandasDF = pd.read_csv(path +  file, sep=sep)
    return pandasDF

In [ ]:
stops_df = getPandasDFCSV(GTFS_DATA_PATH, 'stops.txt')
#stops_df.head()

In [ ]:
ridership_df = getPandasDFCSV(GTFS_DATA_PATH, 'Metro_Transit_Ridership_by_Route_Weekday.csv')
#ridership_df

In [ ]:
stop_times_df = getPandasDFCSV(GTFS_DATA_PATH, 'stop_times.txt')
#stop_times_df.info()
#stop_times_df.head(3)

In [ ]:
# Fix the times from strings and bad formatting to datetime objects:

arrivalTimesArray = np.array(stop_times_df['arrival_time'])

arrivalTimesArrayFixed = []
for time in arrivalTimesArray:
    timeList = time.split(":")
    timeList = [int(time) for time in timeList]
    if timeList[0] < 24:
        dateTime = datetime.datetime(2000, 1, 1, timeList[0], timeList[1], timeList[2])
    else:
        dateTime = datetime.datetime(2000, 1, 2, timeList[0]-24, timeList[1], timeList[2])
    arrivalTimesArrayFixed.append(dateTime)
    
stop_times_df['arrival_time'] = arrivalTimesArrayFixed

departureTimesArray = np.array(stop_times_df['departure_time'])

departureTimesArrayFixed = []
for time in departureTimesArray:
    timeList = time.split(":")
    timeList = [int(time) for time in timeList]
    if timeList[0] < 24:
        dateTime = datetime.datetime(2000, 1, 1, timeList[0], timeList[1], timeList[2])
    else:
        dateTime = datetime.datetime(2000, 1, 2, timeList[0]-24, timeList[1], timeList[2])
    departureTimesArrayFixed.append(dateTime)
    
stop_times_df['departure_time'] = departureTimesArrayFixed

#stop_times_df.head()

In [ ]:
#stop_times_df.head()

In [ ]:
trips_df = getPandasDFCSV(GTFS_DATA_PATH, 'trips.txt')
#trips_df.info()
#trips_df.head(3)

In [ ]:
stops_df = getPandasDFCSV(GTFS_DATA_PATH, 'stops.txt')
#stops_df.info()
#stops_df.head(3)

In [ ]:
print("list of possible trips: \n")

service_id_list = []
for i in trips_df['service_id']:
    if i not in service_id_list:
        service_id_list.append(i)
print(service_id_list)

In [ ]:
print("list of possible routes: \n")

route_short_name_list = []
for i in trips_df['route_short_name']:
    if i not in route_short_name_list:
        route_short_name_list.append(i)

route_short_name_list.sort()
print(route_short_name_list)

In [ ]:
trips_df_weekday_peak = trips_df[trips_df['service_id'] == '92_WKD']
trips_df_weekday_peak_R = trips_df[trips_df['service_id'] == '92_WKD:R']
trips_df_weekday_peak_s = trips_df[trips_df['service_id'] == '92_WKD:L#6']
trips_df_weekday_peak_s3 = trips_df[trips_df['service_id'] == '92_WKD:L=6']
trips_df_weekday_peak_s1 = trips_df[trips_df['service_id'] == '92_WKD:S']
trips_df_weekday_peak_s2 = trips_df[trips_df['service_id'] == '92_WKD:S#6']
trips_df_weekday_peak_s4 = trips_df[trips_df['service_id'] == '92_WKD:S=6']

dfs_wk = [trips_df_weekday_peak, trips_df_weekday_peak_R, trips_df_weekday_peak_s, 
          trips_df_weekday_peak_s1, trips_df_weekday_peak_s2, trips_df_weekday_peak_s4]

trips_df_weekday = pd.concat(dfs_wk)
#trips_df_weekday.info()

In [ ]:
print("Routes for weekday trips: \n")

routes_wk_list = []
for route in trips_df_weekday['route_short_name']:
    if route not in routes_wk_list:
        routes_wk_list.append(route)

routes_wk_list.sort()
print(routes_wk_list)

In [ ]:
# make a dictionary of routes with trip ID's for weekday service:
print("Dictionary of all the possible trips for each route during weekdays: \n\n Example output for 10 trips in route 2: \n")

tripIDList_forRoutes_weekday92 = dict()
for i in routes_wk_list:
    trip_list = []
    for j in trips_df_weekday[trips_df_weekday['route_short_name'] == i].iterrows():
        trip_list.append(j[1]['trip_id'])
    tripIDList_forRoutes_weekday92[i] = trip_list

print(tripIDList_forRoutes_weekday92[2][:10])

# Find average speed for each route: 

total distance travelled by route / time the route took

In [ ]:
ridership_df[ridership_df['Route'] == 7]

totalRiders = np.sum(list(ridership_df['DailyBoardings']))

In [ ]:
# Each route has a list [average speed, average stop spacing, percent of total ridership]

route_stops = dict()

totalRidersAll = np.sum(list(ridership_df['DailyBoardings']))

route_characteristics = dict()
for route in tripIDList_forRoutes_weekday92:
    route_characteristics[route] = []
    route_stops[route] = []
    
    stops_list = []
    route_trips_speeds = []
    route_trips_distances = []
    stop_spacing_all = []
    farthest_distances = []
    for trip in tripIDList_forRoutes_weekday92[route]:
        
        # Find speeeds: 
        stop_times_df_local = stop_times_df[stop_times_df['trip_id'] == trip]
        distance = list(stop_times_df_local['shape_dist_traveled'])[-1]
        time = list(stop_times_df_local['departure_time'])[-1] - list(stop_times_df_local['departure_time'])[0]
        speed = distance / (time.seconds / 3600)
        route_trips_speeds.append(speed)
        
        # Find average route spacing:
        spacing_list = []
        spacing = list(stop_times_df_local['shape_dist_traveled'])
        spacing.insert(0, 0.0)
        for i in range(len(spacing)-1):
            stop_spacing = spacing[i+1] - spacing[i]
            spacing_list.append(stop_spacing)
            stop_spacing_all.append(stop_spacing)
        route_trips_distances.append(np.average(spacing_list))
           
        # Add stops to dictionary:
        for stop in stop_times_df_local['stop_id']:
            if stop not in stops_list:
                stops_list.append(stop)
                
        #farthest distances:
        farthest_distances.append(spacing[-1])
        
    # Get daily boardings:
    route_df = ridership_df[ridership_df['Route'] == route]
    totalRiders = np.sum(list(route_df['DailyBoardings']))
    fractionOfTotalRiders = totalRiders/totalRidersAll
    
    # get the distribution of stop spacings:
    stop_spacing_dict = dict()
    stop_spacing_all.sort()
    for i in stop_spacing_all:
        if i not in stop_spacing_dict:
            stop_spacing_dict[i] = 1
        else:
            stop_spacing_dict[i] += 1
    
    route_characteristics[route].append(np.average(route_trips_speeds))
    route_characteristics[route].append(np.average(route_trips_distances))
    route_characteristics[route].append(fractionOfTotalRiders)
    route_characteristics[route].append(stop_spacing_dict)
    route_characteristics[route].append(np.average(farthest_distances))
    route_stops[route] = stops_list
    

#route_characteristics[27]

## Plotting the Routes and Characteriestics:

In [ ]:
Routes_latLon = dict()

for route in routes_wk_list:
    lat = []
    lon = []
    stops = route_stops[route]
    for stop in stops:
        stops_local = stops_df[stops_df['stop_id'] == stop]
        lat.append(float(stops_local['stop_lat']))
        lon.append(float(stops_local['stop_lon']))
    Routes_latLon[route] = [lat, lon]

In [ ]:
riderships = []
for route in route_characteristics:
    if route == 80: 
        continue
    riderships.append(route_characteristics[route][2])

ridership_avg_no_80 = np.average(riderships)
ridership_avg_no_80

Speeds = []
for route in route_characteristics:
    if route == 80: 
        continue
    Speeds.append(route_characteristics[route][0])

Speeds_avg_no_80 = np.average(Speeds)
Speeds_avg_no_80

spacing = []
for route in route_characteristics:
    if route == 80: 
        continue
    spacing.append(route_characteristics[route][1])

Spacing_avg_no_80 = np.average(spacing)
Spacing_avg_no_80

std_dev = []
for route in route_characteristics:
    if route == 80: 
        continue
    std_dev.append(np.sqrt(np.var(list(route_characteristics[route][3].keys()))))

std_dev_avg_no_80 = np.average(std_dev)
std_dev_avg_no_80

farthest_dist = []
for route in route_characteristics:
    if route == 80: 
        continue
    farthest_dist.append(route_characteristics[route][4])

farthest_dist_avg_no_80 = np.average(farthest_dist)
farthest_dist_avg_no_80

print(ridership_avg_no_80, Speeds_avg_no_80, Spacing_avg_no_80, std_dev_avg_no_80, farthest_dist_avg_no_80)

In [ ]:
maxxx = 0
for route in routes_wk_list:
    dist = list(route_characteristics[route][3].keys())
    if max(dist) > maxxx:
        maxxx = max(dist)
        
maxxx

In [ ]:
colors_uniform = {1: ['#020035'], 2:["#769958"], 3:["#a24857", "#276ab3", "#c9643b"]}

In [ ]:
city = geopandas.read_file("shapes/City")
lakes = geopandas.read_file("shapes/Lakes")
Street = geopandas.read_file("shapes/Street")

In [ ]:
def plotRouteCharacteristics_colors(route):
    ax = plt.subplots(1, 3, figsize=(35, 9), frameon=False, gridspec_kw={'width_ratios': [1.5, 1.8, 1.4]})[1]

    ### Plot 1: Add the geopandas backgroud:
    city.plot(color="lightgray", alpha=.2, ax=ax[0], zorder=2)
    lakes.plot(color="lightblue", ax=ax[0], zorder=1, alpha=.8)
    Street.plot(color="darkgray", alpha = .5, ax=ax[0], zorder=3)

    ### Plot 2: Add the routes on top:
    x = Routes_latLon[route][1]
    y = Routes_latLon[route][0]
    
    # Look for specific colors or default to green
    ax[0].scatter(x, y, label="Route stops", color="#410056", zorder=4, marker='o', s=20)
        
    ax[0].set_xlim(min(Routes_latLon[route][1])-.01, max(Routes_latLon[route][1])+.01)
    ax[0].set_ylim(min(Routes_latLon[route][0])-.005, max(Routes_latLon[route][0])+.005)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].legend()
    ax[0].set_title("Route Map", fontsize=18)

    ### Plot 3: Bar plot to the right summarizing stats
    ridership_perc_avg = route_characteristics[route][2]/ridership_avg_no_80
    speed_perc_avg = route_characteristics[route][0]/Speeds_avg_no_80
    spacing_perc_avg = route_characteristics[route][1]/Spacing_avg_no_80
    std_dev = np.sqrt(np.var(list(route_characteristics[route][3].keys())))/std_dev_avg_no_80
    farthest_dist_avg = route_characteristics[route][4]/farthest_dist_avg_no_80

    x_labels = ["Ridership", "Speed", "Farthest Distance", "Stop Spacing", "Std Deviation \n of Spacing"]
    hist_x = (1, 2, 3, 4, 5)
    width = .8
    hist_heights = (ridership_perc_avg, speed_perc_avg, farthest_dist_avg, spacing_perc_avg, std_dev)
    
    # Look for specific colors or default to green
    ax[1].bar(x=hist_x, height=hist_heights, color=((66/255, 146/255, 141/255), (66/255, 146/255, 141/255), (66/255, 146/255, 141/255), 
                                                    (66/255, 146/255, 141/255), (66/255, 146/255, 141/255)), tick_label=x_labels, 
              width = width, alpha=.8)

    # Place text above bars that too small:
    ylim = max(1, speed_perc_avg+.1, ridership_perc_avg+.1, spacing_perc_avg+.1, std_dev+.1, farthest_dist_avg+.1) 
    if ridership_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("% daily riders:\n", xy=(1, ridership_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.3f} %'.format(route_characteristics[route][2]*100), xy=(1, ridership_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)  
    else:
        ax[1].annotate("% daily riders:\n", xy=(1, ridership_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.3f} %'.format(route_characteristics[route][2]*100), xy=(1, ridership_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)

    if speed_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("Avg Speed:\n", xy=(2, speed_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.1f} MPH'.format(route_characteristics[route][0]), xy=(2, speed_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)
    else:
        ax[1].annotate("Avg Speed:\n", xy=(2, speed_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.1f} MPH'.format(route_characteristics[route][0]), xy=(2, speed_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)
    
    if farthest_dist_avg-.04*ylim < .1*ylim:
        ax[1].annotate("Farthest Dist:\n", xy=(3, farthest_dist_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.2f} Miles'.format(route_characteristics[route][4]), xy=(3, farthest_dist_avg+.02*ylim), ha='center', va='center', fontsize=14)
    else:
        ax[1].annotate("Farthest Dist: \n", xy=(3, farthest_dist_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.2f} Miles'.format(route_characteristics[route][4]), xy=(3, farthest_dist_avg-.06*ylim), ha='center', va='center', fontsize=14)

    if spacing_perc_avg-.04*ylim < .1*ylim:
        ax[1].annotate("Avg Spacing:\n", xy=(4, spacing_perc_avg+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.2f} Miles'.format(route_characteristics[route][1]), xy=(4, spacing_perc_avg+.02*ylim), ha='center', va='center', fontsize=14)
    else:
        ax[1].annotate("Avg Spacing:\n", xy=(4, spacing_perc_avg-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.2f} Miles'.format(route_characteristics[route][1]), xy=(4, spacing_perc_avg-.06*ylim), ha='center', va='center', fontsize=14)
        
    if std_dev-.04*ylim < .1*ylim:
        ax[1].annotate("Std_dev:\n", xy=(5, std_dev+.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.3f} Miles'.format(np.sqrt(np.var(list(route_characteristics[route][3].keys())))), xy=(5, std_dev+.02*ylim), ha='center', va='center', fontsize=14)  
    else:
        ax[1].annotate("Std_dev:\n", xy=(5, std_dev-.04*ylim), ha='center', va='center', fontsize=12)
        ax[1].annotate('{0:^.3f} Miles'.format(np.sqrt(np.var(list(route_characteristics[route][3].keys())))), xy=(5, std_dev-.06*ylim), ha='center', va='center', fontsize=14)

    ax[1].tick_params(labelsize=14)
    ax[1].set_ylabel("Fraction of Average from All Routes", fontsize=16)
    ax[1].set_ylim(0, ylim)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['top'].set_visible(False)
    
    # Make the distribution plots:
    spacing = list(route_characteristics[route][3].keys())
    #print(spacing)
    counts = list(route_characteristics[route][3].values())
    bins = [i*.1 for i in range(math.ceil(10*max(spacing)+1))]
    #print(bins)
    ax[2].hist(route_characteristics[route][3], bins=bins, rwidth=.9, color=(118/255, 203/255, 90/255))
    ax[2].spines['right'].set_visible(False)
    ax[2].spines['top'].set_visible(False)
    ax[2].set_ylabel("Counts (arb)", fontsize=16)
    ax[2].set_xlabel("Stop spacing (Miles)", fontsize=16)
    ax[2].set_yticks([])
    ax[2].tick_params(labelsize=14)
    ax[2].set_title("Distribution of Stop Spacing", fontsize=16)

    plt.suptitle("Route " + str(route) + " Characteristics", fontsize=24)
    #plt.savefig("PaperFigures/"+"Route_" + str(route), bbox_inches='tight')
    plt.savefig("Figures/"+"Route_" + str(route), bbox_inches='tight')
    #plt.savefig("PaperFigures/"+"Route_" + str(route), bbox_inches='tight', format="svg")
    
    plt.show()
    
    

In [ ]:
plotRouteCharacteristics_colors(27)

In [ ]:
for route in routes_wk_list:
    plotRouteCharacteristics_colors(route)